## Huggingface 

In [1]:
# pip install torch --extra-index-url https://download.pytorch.org/whl/cu116
# !pip install transformers
# !pip install ipywidgets

Huggingface에서 제공하는 transformer 모델들은 텍스트를 바로 입력받을 수 없기에, 각 모델별로 tokenizer라는 것을 제공
- 텍스트를 tokenize한 후 각 token들을 고유한 정수로 바꾼 것(token id들의 list ; sequence of token ids)를 입력으로 받는다.
- attention_mask, token_type_ids와 같은 추가적인 데이터를 요구한다. 
- tokenizer : 자신과 짝이 맞는 모델이 어떤 형태의 입력을 요구하는지를 알고있어, 이를 이용하면 텍스트를 전처리, 가공해서 input을 만들 수 있다. 
- 따라서 tokenizer의 출력을 그대로 모델의 input에 넣어주기만 하면 된다. 
    > 모델이 받아들일 수 있는 형태로 입력 텍스트를 가공해 준다. 


In [2]:
# 1단계 ) 해당 모델의 tokenizer 불러오기 
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased') #bert-base-uncased 모델의 tokenizer를 불러온다 

In [8]:
# 각 개체에 접근하는 방법이다. 
tokens = tokenizer("I like like you you", "I love MLP") 

#input ids : 101은 시작토큰, 1045는 I , 2066은 like, 2017은 you로 나타내진다.
#token type ids : sentence가 2문장이므로 token type ids에서 0과 1로 나타나고 ','로 구분한다
#attention_mask : attention 연산이 수행되어야 할 token과 무시해야 할 token을 구별하는 정보가 담긴 list / 연산이 수행될 필요가 없는 token에는 0을 부여한다 

print(f"input ids : {tokens['input_ids']}")
print(f"token type ids : {tokens['token_type_ids']}")
print(f"attention mask : {tokens['attention_mask']}")

input ids : [101, 1045, 2066, 2066, 2017, 2017, 102, 1045, 2293, 19875, 2361, 102]
token type ids : [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
attention mask : [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [10]:
# 각 개체에 접근하는 방법이다. 
tokens = tokenizer("I like like you you", "I love MLP",
                   return_token_type_ids=False,
                   return_attention_mask=False) 
tokens

{'input_ids': [101, 1045, 2066, 2066, 2017, 2017, 102, 1045, 2293, 19875, 2361, 102]}

In [11]:
# tokenizer.convert_ids_to_tokens() 메소드를 사용하면 token id를 token으로 변환할 수 있다.
print(tokenizer.convert_ids_to_tokens(1045))  # 하나만 바꿀 수도 있고
print(tokenizer.convert_ids_to_tokens([101, 1045, 2293, 17953, 2361, 999, 102]))  # 여러 개를 바꿀 수도 있다


# 위 결과를 보면 token들이 모두 소문자임을 볼 수 있다. 
# 이는 bert-base-uncased tokenizer가 이름에 걸맞게 입력된 텍스트를 모두 소문자로 변환한 후 tokenization을 진행하기 때문에 그렇다.
# nl , ##p 롤 구분된 것? BERT tokenizer가 wordpiece 알고리즘에 따라 subword 단위로 token을 분리한다. 
# 101, 102는 special token : [CLS], [SEP]로 사용된다. 즉, 시작할 때, 끝날 때 사용된다. 

i
['[CLS]', 'i', 'love', 'nl', '##p', '!', '[SEP]']


## special token의 정보 

In [17]:
print(f"special token ids : {tokenizer.all_special_ids}")
print(f"special tokens : {tokenizer.all_special_tokens}")

# UNK 토큰은 존재하지 않는 토큰이다.

special token ids : [100, 102, 0, 101, 103]
special tokens : ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']


In [18]:
#special token이 삽입되는 것을 막기위해, False로 설정한다.

tokens = tokenizer(
    "I love NLP!",
    add_special_tokens=False    
)
print(f"token ids : {tokens['input_ids']}")
print(f"tokens : {tokenizer.convert_ids_to_tokens(tokens['input_ids'])}")

token ids : [1045, 2293, 17953, 2361, 999]
tokens : ['i', 'love', 'nl', '##p', '!']


In [19]:
print(tokenizer.convert_tokens_to_ids("I"))  # 하나만 바꿀 수도 있고
print(tokenizer.convert_tokens_to_ids(['[CLS]', 'i', 'love', 'nl', '##p', '!', '[SEP]']))  # 여러 개를 바꿀 수도 있다

# 이 메소드에는 반드시 tokenizer의 vocabulary에 존재하는 token만 입력해야 한다.
# 존재하지 않는 token, 이를테면 대문자 "I" 따위를 변환하려 하면 [UNK] 토큰으로 변환된다.

100
[101, 1045, 2293, 17953, 2361, 999, 102]


In [20]:
tokens = tokenizer("I love NLP!")
print(tokens)
print(f"Decoded : {tokenizer.decode(tokens['input_ids'])}")
print(f"Decoded (skip special tokens) : {tokenizer.decode(tokens['input_ids'], skip_special_tokens=True)}") 
# skip_special_tokens = True는 special token 무시하고 문자열로 변환된다.

{'input_ids': [101, 1045, 2293, 17953, 2361, 999, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1]}


2023-05-25 14:31:04.345875: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-25 14:31:05.854209: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


Decoded : [CLS] i love nlp! [SEP]
Decoded (skip special tokens) : i love nlp!


In [24]:
# 여러 텍스트를 동시에 tokenize할 수도 있다

tokenizer(["I love NLP!","I don't like NLP..."])

{'input_ids': [[101, 1045, 2293, 17953, 2361, 999, 102], [101, 1045, 2123, 1005, 1056, 2066, 17953, 2361, 1012, 1012, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [25]:
tokenizer([
    ["I love NLP!", "I don't like NLP..."],
    ["There is an apple.", "I want to eat it."]
])

{'input_ids': [[101, 1045, 2293, 17953, 2361, 999, 102, 1045, 2123, 1005, 1056, 2066, 17953, 2361, 1012, 1012, 1012, 102], [101, 2045, 2003, 2019, 6207, 1012, 102, 1045, 2215, 2000, 4521, 2009, 1012, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}

In [43]:
tokenizer(
    "I love NLP!",
    return_tensors="pt"
)

{'input_ids': tensor([[  101,  1045,  2293, 17953,  2361,   999,   102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]])}

In [44]:
# 만약 각 텍스트의 token 수가 다르다면 pytorch tensor로 출력을 받을 수 없다. torch tensor의 각 row는 크기가 같아야 하기 때문이다.
tokenizer(
    ["I love NLP!", "I don't like NLP..."],
    return_tensors="pt"
)  # ERROR

ValueError: Unable to create tensor, you should probably activate truncation and/or padding with 'padding=True' 'truncation=True' to have batched tensors with the same length. Perhaps your features (`input_ids` in this case) have excessive nesting (inputs type `list` where type `int` is expected).

##### 본 문서에서는 BERT의 동작 과정을 알아보는 것이 목적이므로 add_pooling_layer에는 False를, output_hidden_states와 output_attentions에는 True를 주었다.

In [45]:
from transformers import BertModel

model = BertModel.from_pretrained("bert-base-uncased", add_pooling_layer=False, output_hidden_states=True, output_attentions=True)

# add_pooling_layer : BERT 위에 추가적으로 pooling layer를 쌓을 것인지 여부를 결정한다(default: True)
# output_hidden_states : BERT의 각 layer의 hidden state들을 담고 있는 배열 hidden_states를 출력할 것인지 여부를 결정
# ouput_attentions : BERT의 각 layer의 attention distribution(attention weight)들을 담고 있는 배열 attentions를 출력할 것인지 여부를 결정

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'bert.pooler.dense.weight', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'bert.pooler.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [46]:
model.num_parameters() ## huggingface의 bert 모델은 1080M개의 파라미터로 이루어져 있다.

108891648

In [47]:
model

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

In [51]:
model_input = tokenizer("I love NLP!", return_tensors="pt")
output = model(**model_input)

output.keys()

odict_keys(['last_hidden_state', 'hidden_states', 'attentions'])

In [52]:
print(output.last_hidden_state.shape)

# bert-base-uncased의 경우 (batch_size, sequence_length, 768) 크기의 tensor이다. 일반적으로 이 값을 입력된 텍스트에 대해 BERT가 생성한 최종 embedding으로 여긴다. 

torch.Size([1, 7, 768])


In [59]:
print(len(output.hidden_states))
print(output.hidden_states[-1] == output.last_hidden_state)
print(output.hidden_states[0].shape)

#hidden_states는 각 layer의 hidden state를 모아놓은 list이다. 
# 이때 마지막 layer일수록 뒤에 있다. 즉 hidden_states[-1]과 last_hidden_state는 같다. 
# bert-base-uncased의 경우 길이 13인 list이고(첫 번째 원소는 BertEmbeddings 모듈의 출력값이다), 각 원소는 크기 (batch_size, sequence_length, 768)인 tensor이다.

13
tensor([[[True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         ...,
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True],
         [True, True, True,  ..., True, True, True]]])
torch.Size([1, 7, 768])


In [60]:
print(len(output.attentions))
print(output.attentions[0].shape)

# attentions은 각 layer의 attention weight를 모아놓은 list이다. 이때 마지막 layer일수록 뒤에 있다. 
# bert-base-uncased의 경우 길이 12인 list이고, 각 원소는 크기 (batch_size, 12, sequence_length, sequence_length)인 tensor이다.

12
torch.Size([1, 12, 7, 7])


In [66]:
model.embeddings.token_type_embeddings
# 여기서 2는 BERT가 받아들일 수 있는 sentence 개수(sentence A, sentence B)를 의미하고, 768은 마찬가지로 hidden_size(출력되는 embedding의 차원)를 의미한다.

Embedding(2, 768)

https://heekangpark.github.io/nlp/huggingface-bert 